In [1]:
import os

from hostdesigner import host_export, linker_export, drive_export

main_dir = os.getcwd()
hd_dir = os.path.join(main_dir, 'HD_3.0')
examples_dir = os.path.join(hd_dir, '03_Examples')

results_dir = os.path.join(main_dir, 'results')

In [3]:
case1_dir = os.path.join(examples_dir, 'case1')
hosta_dir = os.path.join(case1_dir, 'hosta')

export_dir = os.path.join(results_dir, 'case1')
if not os.path.exists(export_dir):
    os.mkdir(export_dir)

host_export(hosta_dir, export_dir)

linker_path = os.path.join(case1_dir, 'out_1.hdo')
linker_export(linker_path, export_dir, num=10, xyz_name='_Case1_L')

In [5]:
case2_dir = os.path.join(examples_dir, 'case2')
lilig_dir = os.path.join(case2_dir, 'Li_lig')

export_dir = os.path.join(results_dir, 'case2')
if not os.path.exists(export_dir):
    os.mkdir(export_dir)

host_export(lilig_dir, export_dir)

linker_path = os.path.join(case2_dir, 'case2_1.hdo')
linker_export(linker_path, export_dir, num=10, xyz_name='_Case2_1_L')

linker_path = os.path.join(case2_dir, 'case2_2.hdo')
linker_export(linker_path, export_dir, num=10, xyz_name='_Case2_2_L')

In [3]:
case3_dir = os.path.join(examples_dir, 'case3')
urea1_dir = os.path.join(case3_dir, 'urea1')
urea2_dir = os.path.join(case3_dir, 'urea2')

export_dir = os.path.join(results_dir, 'case3')
if not os.path.exists(export_dir):
    os.mkdir(export_dir)

host_export(urea1_dir, export_dir)
host_export(urea2_dir, export_dir)


drive_path = os.path.join(case3_dir, 'out_testa.hdo')
drive_export(drive_path, export_dir, num=10, xyz_name='_Case3_A_L')

drive_path = os.path.join(case3_dir, 'out_testb.hdo')
drive_export(drive_path, export_dir, num=10, xyz_name='_Case3_B_L')

In [5]:
case5_dir = os.path.join(examples_dir, 'case5')
over1_dir = os.path.join(case5_dir, 'over1')

host_export(over1_dir, export_dir)

In [6]:
case6_dir = os.path.join(examples_dir, 'case6')
over2_dir = os.path.join(case6_dir, 'over2')

host_export(over2_dir, export_dir)

In [7]:
case7_dir = os.path.join(examples_dir, 'case7')
tetrainp_dir = os.path.join(case7_dir, 'tetra_inp')

host_export(tetrainp_dir, export_dir)

In [8]:
case8_dir = os.path.join(examples_dir, 'case8')
ureamacro_dir = os.path.join(case8_dir, 'urea_macro')

host_export(ureamacro_dir, export_dir)

In [ ]:
linker_path = os.path.join(case1_dir, 'out_1.hdo')

linker_export(linker_path, export_dir, num=15)

In [2]:
os.isdir(export_dir)

AttributeError: module 'os' has no attribute 'isdir'